In [1]:
import pandas as pd
import numpy as np
from glob import glob
from scipy import stats
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
import json
import random
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
from transformer_lens import HookedTransformer
import torch
from jaxtyping import Float, Int
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD
import itertools
import torch.nn.functional as F
import math
import seaborn as sns

## Setup

In [2]:
def get_device_str() -> str:
    if torch.backends.mps.is_available():
        return "mps"
    else:
        return "cuda" if torch.cuda.is_available() else "cpu"


def comp_js_divergence(
    p_logit: Float[torch.Tensor, "*batch vocab"],
    q_logit: Float[torch.Tensor, "*batch vocab"],
) -> Float[torch.Tensor, "*batch"]:
    p_logprob = torch.log_softmax(p_logit, dim=-1)
    q_logprob = torch.log_softmax(q_logit, dim=-1)
    p = p_logprob.exp()
    q = q_logprob.exp()

    # convert to log2
    p_logprob *= math.log2(math.e)
    q_logprob *= math.log2(math.e)

    m = 0.5 * (p + q)
    m_logprob = m.log2()

    p_kl_div = (p * (p_logprob - m_logprob)).sum(-1)
    q_kl_div = (q * (q_logprob - m_logprob)).sum(-1)

    assert p_kl_div.isfinite().all()
    assert q_kl_div.isfinite().all()
    return (p_kl_div + q_kl_div) / 2

In [3]:
model = HookedTransformer.from_pretrained("gpt2")
device = get_device_str()

print(device)

Loaded pretrained model gpt2 into HookedTransformer
cuda


In [4]:
perturbation_layer = "blocks.0.hook_resid_pre"
saes, sparsities = get_gpt2_res_jb_saes(perturbation_layer)
sae = saes[perturbation_layer].cpu()
feature_sparsities = 10 ** sparsities[perturbation_layer].cpu()

sae.eval()

100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


SAE(
  (activation_fn): ReLU()
  (hook_sae_input): HookPoint()
  (hook_sae_acts_pre): HookPoint()
  (hook_sae_acts_post): HookPoint()
  (hook_sae_output): HookPoint()
  (hook_sae_recons): HookPoint()
  (hook_sae_error): HookPoint()
)

In [5]:
def load_jsonl(file_path):
    data = []
    with open(file_path, "r") as file:
        for line in file:
            # Parse each line as a JSON object
            json_object = json.loads(line.strip())
            data.append(json_object)
    return data


def find_plateau_length(
    steps: List[Dict], threshold: float = 0.005, window_size: int = 5
) -> int:
    kl_divs = [step["kl_div"] for step in steps]

    def calculate_slope(y_values, x_values):
        slope, _, _, _, _ = stats.linregress(x_values, y_values)
        return slope

    for i in range(len(kl_divs) - window_size + 1):
        window = kl_divs[i : i + window_size]
        x_values = list(range(window_size))
        slope = calculate_slope(window, x_values)

        if abs(slope) > threshold:
            return i

    return len(steps) - 1


def find_plateau_length_js(
    steps: List[Dict], threshold: float = 5e-05, window_size: int = 5
) -> int:
    js_divs = [step["js_div"] for step in steps]

    def calculate_slope(y_values, x_values):
        slope, _, _, _, _ = stats.linregress(x_values, y_values)
        return slope

    for i in range(len(js_divs) - window_size + 1):
        window = js_divs[i : i + window_size]
        x_values = list(range(window_size))
        slope = calculate_slope(window, x_values)

        if abs(slope) > threshold:
            return i

    return len(steps) - 1


def get_feature_dir(sae, feature_id, feature_act, norm=False):
    feature_acts = torch.zeros(sae.W_enc.shape[1])

    # Activate the specified feature
    feature_acts[:] = 0
    feature_acts[feature_id] = feature_act

    # Decode back to the residual stream space
    perturbed_act = sae.decode(feature_acts)

    if norm:
        perturbed_act /= torch.norm(perturbed_act)

    return perturbed_act


def find_js_dist_blowup(
    js_dists: List[float],
    slope_threshold: float = 1.25,
    min_js_distance: float = 0.1,
    min_slope_threshold: float = 0.02,
) -> int:
    steps = list(range(len(js_dists)))

    slopes = []
    for i in range(1, len(steps)):
        dy = js_dists[i] - js_dists[i - 1]
        dx = steps[i] - steps[i - 1]
        slopes.append(dy / dx if dx != 0 else 0)

    # Pad the beginning of slopes list to match the length of steps
    slopes = [slopes[0]] + slopes

    for i in range(1, len(steps)):
        current_slope = slopes[i]
        prev_slope = slopes[i - 1]

        # print(f"step={i}, {current_slope=} {prev_slope=}, {slope_threshold * max(prev_slope, 1e-6)}")

        # Check if we're in an increasing trend and above minimum JS distance
        if (
            current_slope > (slope_threshold * max(prev_slope, 1e-6))
            or current_slope >= min_slope_threshold
        ) and js_dists[i] >= min_js_distance:
            return i

    return len(steps) - 1

In [6]:
metadata, activations = [], []

for path in glob("observations/metadata*"):
    metadata.extend(load_jsonl(path))

for path in glob("observations/acts*"):
    activations.append(np.load(path))

activations = np.concatenate(activations)

In [7]:
len(metadata), activations.shape

(5000, (5000, 100, 768))

## Individual feature dampening

In [18]:
dampening_by_rank = defaultdict(lambda: defaultdict(list))

for i, (m, acts) in enumerate(zip(metadata, activations)):
    start_prompt = m["start_prompt"]

    with torch.no_grad():
        start_prompt_gpu = torch.tensor(start_prompt).to(device)
        start_logits, cache = model.run_with_cache(start_prompt_gpu)

    active_feature_ids = m["steps_metadata"][0]["active_feature_ids"]
    active_feature_acts = m["steps_metadata"][0]["active_feature_acts"]

    sorted_feature_ids_with_acts = sorted(
        zip(active_feature_ids, active_feature_acts),
        key=lambda t: t[1],
        reverse=True,
    )

    feature_acts = torch.zeros(sae.W_enc.shape[1])
    feature_acts[active_feature_ids] = torch.tensor(active_feature_acts)

    for rank, (feature_id, feature_act) in enumerate(sorted_feature_ids_with_acts[:5]):
        fa = feature_acts.clone()
        original_feature_act = fa[feature_id]
        feature_act_steps = np.linspace(original_feature_act, 0, 100)

        for step in range(100):
            fa[feature_id] = 0
            ablation_act = sae.decode(fa)

            def hook(act, hook):
                act[:, -1, :] = feature_act_steps[step]

            with model.hooks(fwd_hooks=[(perturbation_layer, hook)]):
                pert_logits, _ = model.run_with_cache(start_prompt_gpu)

            js_div = comp_js_divergence(pert_logits[:, -1, :], start_logits[:, -1, :])
            js_dist = torch.sqrt(js_div + 1e-8).item()
            dampening_by_rank[rank + 1][step].append(js_dist)

    if i % 10 == 0:
        print(f"Processed {i} perturbations")

Processed 0 perturbations
Processed 10 perturbations


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fad9596f370>>
Traceback (most recent call last):
  File "/home/paperspace/virtualenvs/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fad9596f370>>
Traceback (most recent call last):
  File "/home/paperspace/virtualenvs/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
dampening_by_rank

(100,)